### Incializando o solr:

In [65]:
import subprocess

# Comando que você deseja executar
command = 'C:\\Users\\thgcn\\OneDrive\\Academico\\PO-245\\Projeto\\solr-9.2.1\\bin\\solr restart -port 8983'
command2 = 'C:\\Users\\thgcn\\OneDrive\\Academico\\PO-245\\Projeto\\solr-9.2.1\\bin\\solr -c -z localhost:9983 -p 8984'

# Executa o comando no terminal
subprocess.run(command, shell=True)
subprocess.run(command2, shell=True)


CompletedProcess(args='C:\\Users\\thgcn\\OneDrive\\Academico\\PO-245\\Projeto\\solr-9.2.1\\bin\\solr -c -z localhost:9983 -p 8984', returncode=1)

### Criando uma coleção e adcionando os campos para catalogação

In [42]:
import os
import requests

# Caminho para o diretório bin do Solr
solr_bin_path = 'C:\\Users\\thgcn\\OneDrive\\Academico\\PO-245\\Projeto\\solr-9.2.1\\bin\\'

# Comando para criar a coleção no modo standalone
command = 'solr create_collection -c relatoriosfinanceiros -s 2 -rf 2'

# Mude para o diretório bin do Solr
os.chdir(solr_bin_path)

# Execute o comando no terminal
os.system(command)

# Após a criação do seleção, é atualizado os campos que serão atualizados durante a catalogação:

import requests

# URL do endpoint Solr
url = 'http://localhost:8983/api/cores/relatoriosfinanceiros/schema'

# Cabeçalhos da solicitação POST
headers = {
    'Content-Type': 'application/json'
}

# Corpo da solicitação POST
data = {
    "add-field": [
        {"name": "report_id", "type": "string"},
        {"name": "title", "type": "text_general", "multiValued": False},
        {"name": "company_name", "type": "text_general", "multiValued": False},
        {"name": "cod_cvm", "type": "text_general", "multiValued": False},
        {"name": "year", "type": "pint"},
        {"name": "date", "type": "pdate"},
        {"name": "content", "type": "text_general", "multiValued": True},
        {"name": "keywords", "type": "text_general", "multiValued": True},
        {"name": "size", "type": "text_general", "multiValued": True},
        {"name": "tensor", "type": "text_general","multiValued": True}
    ]
}

# Enviar solicitação POST para atualizar o esquema no Solr
response = requests.post(url, json=data, headers=headers)

if response.status_code == 200:
    print("Esquema atualizado com sucesso.")
else:
    print("Falha ao atualizar o esquema. Status:", response.status_code)


Esquema atualizado com sucesso.


### Catalotação de relatórios

In [43]:
import requests
import glob
from io import BytesIO
from PyPDF2 import PdfFileReader
import json

# URL do endpoint Solr
url = 'http://localhost:8983/solr/relatorios/update/json/docs'

# Parâmetros da solicitação POST
params = {
    'commit': 'true'
}

# Percorra os arquivos PDF no diretório
files = glob.glob('C:\\Users\\thgcn\\OneDrive\\Academico\\PO-245\\Projeto\\financas-quantitativas\\datasets\\19348_ITAU_UNIBANCO_HOLDING_S_A\\2022\\Demonstrações_Financeiras_em_Padrões_Internacionais\\*.pdf')

# Enviar solicitação POST para cada arquivo PDF
for file in files:
    with open(file, 'rb') as f:
        pdf_content = f.read()

        # Converter o conteúdo PDF em texto
        pdf = PdfFileReader(BytesIO(pdf_content))
        text = ''
        for page_num in range(pdf.getNumPages()):
            page = pdf.getPage(page_num)
            text += page.extract_text()

        # Criar um documento JSON com o texto extraído
        document = {
            'text': text
        }

        # Converter o documento para uma string JSON
        json_document = json.dumps(document)

        # Enviar solicitação POST para o Solr
        response = requests.post(url, params=params, data=json_document.encode('utf-8'), headers={'Content-type': 'application/json'})

        if response.status_code == 200:
            print(f'Sucesso ao enviar o arquivo PDF {file}')
        else:
            print(f'Falha ao enviar o arquivo PDF {file}. Status:', response.status_code)


Sucesso ao enviar o arquivo PDF C:\Users\thgcn\OneDrive\Academico\PO-245\Projeto\financas-quantitativas\datasets\19348_ITAU_UNIBANCO_HOLDING_S_A\2022\Demonstrações_Financeiras_em_Padrões_Internacionais\9348_ITA__UNIBANCO_HOLDING_S_A__ers_o_em_Ingl_s_2022_02_10.pdf
Sucesso ao enviar o arquivo PDF C:\Users\thgcn\OneDrive\Academico\PO-245\Projeto\financas-quantitativas\datasets\19348_ITAU_UNIBANCO_HOLDING_S_A\2022\Demonstrações_Financeiras_em_Padrões_Internacionais\9348_ITA__UNIBANCO_HOLDING_S_A__ers_o_em_Ingl_s_2022_08_08.pdf
Sucesso ao enviar o arquivo PDF C:\Users\thgcn\OneDrive\Academico\PO-245\Projeto\financas-quantitativas\datasets\19348_ITAU_UNIBANCO_HOLDING_S_A\2022\Demonstrações_Financeiras_em_Padrões_Internacionais\9348_ITA__UNIBANCO_HOLDING_S_A__ers_o_em_Portugu_s_2022_02_10.pdf
Sucesso ao enviar o arquivo PDF C:\Users\thgcn\OneDrive\Academico\PO-245\Projeto\financas-quantitativas\datasets\19348_ITAU_UNIBANCO_HOLDING_S_A\2022\Demonstrações_Financeiras_em_Padrões_Internacionais\

In [67]:
import requests

def search_name(name):
    # URL do endpoint Solr
    url = 'http://localhost:8983/solr/relatorios/select'

    # Parâmetros da consulta
    params = {
        'q': f'name:{name}'
    }

    # Enviar solicitação GET para o Solr
    response = requests.get(url, params=params)

    if response.status_code == 200:
        # Processar os resultados da consulta
        data = response.json()
        # Faça algo com os resultados aqui
        print(data)
    else:
        print('Falha ao consultar o Solr. Status:', response.status_code)

# Exemplo de uso da função
search_name('itau')


Falha ao consultar o Solr. Status: 400


In [62]:
def search_name(name):
    # URL do endpoint Solr (verifique se está correta, incluindo o nome da coleção)
    url = 'http://localhost:8983/solr/relatorios/select'

    # Parâmetros da consulta (verifique se o campo e o valor estão corretos)
    params = {
        'q': f'name:{name}'
    }

    # Enviar solicitação GET para o Solr
    response = requests.get(url, params=params)

    if response.status_code == 200:
        # Processar os resultados da consulta
        data = response.json()
        # Faça algo com os resultados aqui
        print(data)
    else:
        print('Falha ao consultar o Solr. Status:', response.status_code)

# Exemplo de uso da função
search_name('relatorio')


Falha ao consultar o Solr. Status: 400
